# Simulations

In this notebook, we perform simulations. We compare the coefficient before and after the simulation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
#import xgboost
from xgboost import XGBRegressor
import joblib
#import ptitprince as pt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Import and Processing + Model Import

In [2]:
data = pd.read_csv('data/data.csv')
data['Clim_zone'] = data['Clim_zone'].astype('category')
data['Coastal?'] = data['Coastal?'].astype('category')
data['Median Income (K)'] = data['Median Income'] / 1000
urban_df = data[['Urban_geoid', 'Urban_name', 'Clim_zone', 'Coastal?']].drop_duplicates().copy()

<ipython-input-2-c1d1e846a6c4>:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/data.csv')


In [3]:
model_cuhi_d = joblib.load('models/CUHI_day_summer_xgb_estimator.joblib') 
model_cuhi_n = joblib.load('models/CUHI_night_summer_xgb_estimator.joblib') 
model_suhi_d = joblib.load('models/SUHI_day_summer_xgb_estimator.joblib') 
model_suhi_n = joblib.load('models/SUHI_night_summer_xgb_estimator.joblib') 

In [4]:
features = ['Built Area (%)', 'Grass Area (%)', 'Tree Area (%)', 
            'Albedo_Built', 'Albedo_Grass', 'Albedo_Tree', 
            'DEM_urb_CT_act', 'Clim_zone', 'Coastal?']
labels   = ['CUHI_day_summer', 'CUHI_night_summer', 'SUHI_day_summer', 'SUHI_night_summer']

### Simulation Helper Functions

In [5]:
def get_sim_results(X_test): 
    y_pred_cuhi_d = model_cuhi_d.predict(X_test[features])
    y_pred_cuhi_n = model_cuhi_n.predict(X_test[features])
    y_pred_suhi_d = model_suhi_d.predict(X_test[features])
    y_pred_suhi_n = model_suhi_n.predict(X_test[features])
    X_test['y_pred_cuhi_d'] = y_pred_cuhi_d
    X_test['y_pred_cuhi_n'] = y_pred_cuhi_n
    X_test['y_pred_suhi_d'] = y_pred_suhi_d
    X_test['y_pred_suhi_n'] = y_pred_suhi_n
    return X_test

In [6]:
def apply_regression(df):
    labels = ['y_pred_cuhi_d', 'y_pred_cuhi_n', 'y_pred_suhi_d', 'y_pred_suhi_n']
    X = df['Median Income (K)']
    X = np.array(X).reshape(-1, 1)
    model_cuhi_day = LinearRegression().fit(X, df[labels[0]])
    model_cuhi_night = LinearRegression().fit(X, df[labels[1]])
    model_suhi_day = LinearRegression().fit(X, df[labels[2]])
    model_suhi_night = LinearRegression().fit(X, df[labels[3]])
    return pd.Series({'slope_CUHI_day': model_cuhi_day.coef_[0], 
                      'slope_CUHI_night': model_cuhi_night.coef_[0], 
                      'slope_SUHI_day': model_suhi_day.coef_[0], 
                      'slope_SUHI_night': model_suhi_night.coef_[0]})
    #return pd.Series({'intercept': model.intercept_, 'slope': model.coef_[0]})

In [7]:
def get_coef_results(sim_results):
    coef_result = sim_results.groupby('Urban_geoid').apply(apply_regression)
    coef_result.reset_index(inplace=True)
    coef_result = coef_result.merge(urban_df, how='left', on='Urban_geoid')
    return coef_result

In [8]:
def apply_quantile(df):
    labels  = ['y_pred_cuhi_d', 'y_pred_cuhi_n', 'y_pred_suhi_d', 'y_pred_suhi_n']
    cols    = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']

    med25 = 'Median Income (25th percentile)'
    med75 = 'Median Income (75th percentile)'
    # Calculate the 25th and 75th percentile of median income by urban area
    grouped25 = pd.DataFrame({med25: df.groupby(['Urban_name'])['Median Income'].quantile(0.25)}).reset_index()
    grouped75 = pd.DataFrame({med75: df.groupby(['Urban_name'])['Median Income'].quantile(0.75)}).reset_index()
    # Add a column for the 25th and 75th percentile of median income by urban area
    df = df.merge(grouped25, on='Urban_name')
    df = df.merge(grouped75, on='Urban_name')
    # Compare tract median income with urban area 25th and 75th percentile median income
    df[med25 + '?'] = df['Median Income'] <= df[med25]
    df[med75 + '?'] = df['Median Income'] >= df[med75]
    #display(df.head())

    result = urban_df.copy().reset_index().drop(columns='index')
    for i in range(4):    
        # Filter data points at or below Q1 and Q3, and then calculate the median predicted UHI by urban area
        col25 = cols[i] + ' (25th %tile)'
        col75 = cols[i] + ' (75th %tile)'
        m25 = pd.DataFrame({col25: df[df[med25 + '?'] == True].groupby(['Urban_name'])[labels[i]].median()}).reset_index()  
        m75 = pd.DataFrame({col75: df[df[med75 + '?'] == True].groupby(['Urban_name'])[labels[i]].median()}).reset_index()
        # Update the results
        result = result.merge(m25, on='Urban_name')
        result = result.merge(m75, on='Urban_name')
        result[cols[i] + ' (Difference)'] = result[col75] - result[col25]
    #display(result.head())
        
    return result

## Baseline

In [9]:
X_base = data.dropna().copy()

In [10]:
sim_results_baseline = get_sim_results(X_base)

In [11]:
sim_results_baseline.to_csv('tables/sim_output/baseline.csv', index=False)

In [12]:
coef_result_baseline = get_coef_results(sim_results_baseline)

## Simulation 1: Cool Roofs and Reflective Pavement
To simulate cool roofs and reflective pavements, we set the built albedo to 0.80.

In [13]:
len(data[data['Albedo_Built']>= 0.8])

5

In [14]:
X_albedo = data.dropna().copy()
X_albedo.loc[:, 'Albedo_Built'] = 0.8

In [15]:
sim_results_albedo = get_sim_results(X_albedo)

In [16]:
sim_results_albedo.to_csv('tables/sim_output/albedo.csv', index=False)

In [17]:
coef_result_albedo = get_coef_results(sim_results_albedo)

## Simulation 2: Afforestation

We simulate 50% to 100% conversion of grassland to trees. For full conversion of grassland to trees, the grass albedo is replaced by tree albedo as well.

### 50 percent afforestation

In [18]:
X_afforest_50 = data.dropna().copy()
X_afforest_50['Tree Area (%)']  = X_afforest_50['Tree Area (%)'] + (50/100 * X_afforest_50['Grass Area (%)']) # add the grass area to the tree area
X_afforest_50['Grass Area (%)'] = 50/100 * X_afforest_50['Grass Area (%)'] # divide the grass area by percent afforested area
X_afforest_50['Albedo_Grass']  = X_afforest_50['Albedo_Grass'] * X_afforest_50['Grass Area (%)'] + X_afforest_50['Albedo_Tree'] * X_afforest_50['Tree Area (%)']


In [19]:
sim_results_afforest_50 = get_sim_results(X_afforest_50)

In [20]:
sim_results_afforest_50.to_csv('tables/sim_output/afforest_50.csv', index=False)

In [21]:
coef_result_afforest_50 = get_coef_results(sim_results_afforest_50)

### 100 percent afforestation

In [22]:
X_afforest_100 = data.dropna().copy()
X_afforest_100['Tree Area (%)']  = X_afforest_100['Tree Area (%)'] + X_afforest_100['Grass Area (%)'] # add the grass area to the tree area
X_afforest_100['Grass Area (%)'] = 0 # Subtract 100 percent of grass area from grass area
X_afforest_100['Albedo_Grass']   = X_afforest_100['Albedo_Tree'] # The albedo of the original grass part is now that of trees

In [23]:
sim_results_afforest_100 = get_sim_results(X_afforest_100)

In [24]:
sim_results_afforest_100.to_csv('tables/sim_output/afforest_100.csv', index=False)

In [25]:
coef_result_afforest_100 = get_coef_results(sim_results_afforest_100)

## Simulation 3: Afforestation + Albedo Management

### Cool Roofs/Reflective Pavement + 50 percent Afforestation

In [26]:
X_albedo_afforest_50 = data.dropna().copy()
X_albedo_afforest_50['Tree Area (%)']  = X_albedo_afforest_50['Tree Area (%)'] + (50/100 * X_albedo_afforest_50['Grass Area (%)']) # add the grass area to the tree area
X_albedo_afforest_50['Grass Area (%)'] = 50/100 * X_albedo_afforest_50['Grass Area (%)'] # divide the grass area by percent afforested area
X_albedo_afforest_50['Albedo_Grass']  = X_albedo_afforest_50['Albedo_Grass'] * X_albedo_afforest_50['Grass Area (%)'] + X_albedo_afforest_50['Albedo_Tree'] * X_albedo_afforest_50['Tree Area (%)']
X_albedo_afforest_50.loc[:, 'Albedo_Built'] = 0.8

In [27]:
sim_results_albedo_afforest_50 = get_sim_results(X_albedo_afforest_50)

In [28]:
sim_results_albedo_afforest_50.to_csv('tables/sim_output/albedo_afforest_50.csv', index=False)

In [29]:
coef_result_albedo_afforest_50 = get_coef_results(sim_results_albedo_afforest_50)

### Cool Roofs/Reflective Pavement + 100 percent Afforestation

In [30]:
X_albedo_afforest_100 = data.dropna().copy()
X_albedo_afforest_100['Tree Area (%)']  = X_albedo_afforest_100['Tree Area (%)'] + X_albedo_afforest_100['Grass Area (%)'] # add the grass area to the tree area
X_albedo_afforest_100['Grass Area (%)'] = 0 # Subtract 100 percent of grass area from grass area
X_albedo_afforest_100['Albedo_Grass']  = X_albedo_afforest_100['Albedo_Tree'] # The albedo of the original grass part is now that of trees
X_albedo_afforest_100.loc[:, 'Albedo_Built'] = 0.8

In [31]:
sim_results_albedo_afforest_100 = get_sim_results(X_albedo_afforest_100)

In [32]:
sim_results_albedo_afforest_100.to_csv('tables/sim_output/albedo_afforest_100.csv', index=False)

In [33]:
coef_result_albedo_afforest_100 = get_coef_results(sim_results_albedo_afforest_100)

## Results

### Predicted Urban Heat Intensity by First and Third Quartile of Median Income

In [34]:
iqr_results_baseline = apply_quantile(sim_results_baseline)
iqr_results_baseline.to_csv('tables/quartiles/baseline.csv', index=False)
iqr_results_baseline.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,0.153134,0.128629,-0.024505,0.170664,-0.069281,-0.239945,0.662531,1.043478,0.380947,0.403856,0.140264,-0.263592
1,955,"Albany, OR",Temperate,False,0.384694,-0.025450,-0.410144,0.729594,-0.048602,-0.778196,3.763031,-0.546957,-4.309988,2.004197,0.147296,-1.856902
2,1171,"Albuquerque, NM",Arid,False,0.444491,-0.266217,-0.710707,0.903729,0.512427,-0.391302,1.153673,0.258035,-0.895638,1.527913,0.797699,-0.730213
3,1279,"Alexandria, LA",Temperate,False,0.220576,-0.057764,-0.278339,0.257233,-0.017049,-0.274282,2.810638,-1.183124,-3.993762,0.583172,-0.206620,-0.789792
4,1765,"Alton, IL--MO",Temperate,False,0.152763,-0.154135,-0.306898,0.388192,-0.270974,-0.659166,1.704939,-1.714499,-3.419438,0.975970,-0.773880,-1.749850


In [35]:
iqr_results_albedo = apply_quantile(sim_results_albedo)
iqr_results_albedo.to_csv('tables/quartiles/albedo.csv', index=False)
iqr_results_albedo.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,0.162795,0.161378,-0.001417,0.151109,-0.057171,-0.208280,0.663739,0.818789,0.155050,0.337025,0.057797,-0.279228
1,955,"Albany, OR",Temperate,False,0.636371,-0.027931,-0.664301,0.375496,-0.090433,-0.465929,4.332047,-1.899065,-6.231112,1.016859,-0.168999,-1.185858
2,1171,"Albuquerque, NM",Arid,False,0.423927,-0.181254,-0.605181,0.882564,0.552228,-0.330336,0.758267,-0.368950,-1.127216,1.468807,1.204530,-0.264278
3,1279,"Alexandria, LA",Temperate,False,0.167031,-0.050784,-0.217815,0.184588,-0.018680,-0.203268,2.032720,-2.070722,-4.103441,0.432161,-0.408970,-0.841131
4,1765,"Alton, IL--MO",Temperate,False,0.085884,0.004831,-0.081053,0.314660,-0.165071,-0.479731,1.184185,-2.488072,-3.672257,0.757281,-0.434516,-1.191797


In [36]:
iqr_results_afforest_50 = apply_quantile(sim_results_afforest_50)
iqr_results_afforest_50.to_csv('tables/quartiles/afforest_50.csv', index=False)
iqr_results_afforest_50.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,-0.123313,-0.108452,0.014861,0.376315,-0.058619,-0.434934,0.558942,-0.121204,-0.680146,0.835767,0.337198,-0.498568
1,955,"Albany, OR",Temperate,False,0.538272,-0.323440,-0.861713,0.932503,0.502550,-0.429953,2.101770,-0.937282,-3.039052,0.709687,1.381891,0.672204
2,1171,"Albuquerque, NM",Arid,False,0.307523,-0.168367,-0.475891,0.851673,0.515184,-0.336489,0.163990,-0.557179,-0.721169,1.721693,1.198214,-0.523478
3,1279,"Alexandria, LA",Temperate,False,-0.327576,-0.210082,0.117494,0.116037,0.134947,0.018910,1.170917,-1.738778,-2.909695,0.554399,0.114784,-0.439616
4,1765,"Alton, IL--MO",Temperate,False,-0.483584,-0.142128,0.341456,0.321068,0.178703,-0.142365,-0.239262,-1.158621,-0.919359,0.936259,-0.023092,-0.959351


In [37]:
iqr_results_afforest_100 = apply_quantile(sim_results_afforest_100)
iqr_results_afforest_100.to_csv('tables/quartiles/afforest_100.csv', index=False)
iqr_results_afforest_100.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,-0.823951,-0.930507,-0.106557,0.267942,0.110629,-0.157313,1.189671,0.904361,-0.285310,0.643192,0.572784,-0.070407
1,955,"Albany, OR",Temperate,False,0.557265,0.045721,-0.511544,0.654441,0.478022,-0.176419,3.083219,-1.945077,-5.028296,1.161774,0.993524,-0.168250
2,1171,"Albuquerque, NM",Arid,False,0.250989,-0.104064,-0.355053,0.735700,0.602630,-0.133070,0.510569,-0.170911,-0.681480,0.972416,0.915262,-0.057154
3,1279,"Alexandria, LA",Temperate,False,-0.059904,0.046904,0.106808,0.324874,0.180651,-0.144223,0.434098,-1.644809,-2.078907,0.559290,0.683269,0.123980
4,1765,"Alton, IL--MO",Temperate,False,-1.710661,-0.176343,1.534318,0.244044,0.406948,0.162904,1.230252,-2.164355,-3.394607,0.911145,0.585856,-0.325289


In [38]:
iqr_results_albedo_afforest_50 = apply_quantile(sim_results_albedo_afforest_50)
iqr_results_albedo_afforest_50.to_csv('tables/quartiles/albedo_afforest_50.csv', index=False)
iqr_results_albedo_afforest_50.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,-0.112762,-0.104346,0.008416,0.376129,0.112778,-0.263351,0.545437,-0.018575,-0.564012,0.810124,0.370087,-0.440037
1,955,"Albany, OR",Temperate,False,0.235392,-0.750238,-0.985630,0.482475,0.359114,-0.123362,0.606134,-1.553777,-2.159911,0.513404,-0.048446,-0.561850
2,1171,"Albuquerque, NM",Arid,False,0.363237,-0.246607,-0.609844,0.895947,0.724934,-0.171013,-0.002019,-1.041001,-1.038982,1.547626,1.315445,-0.232181
3,1279,"Alexandria, LA",Temperate,False,-0.216856,-0.188894,0.027962,0.186802,-0.067276,-0.254077,0.842268,-1.553317,-2.395585,0.405514,-0.119651,-0.525165
4,1765,"Alton, IL--MO",Temperate,False,-0.103917,-0.225471,-0.121554,0.331672,0.206132,-0.125539,0.918670,-0.812698,-1.731368,0.844953,-0.285551,-1.130505


In [39]:
iqr_results_albedo_afforest_100 = apply_quantile(sim_results_albedo_afforest_100)
iqr_results_albedo_afforest_100.to_csv('tables/quartiles/albedo_afforest_100.csv', index=False)
iqr_results_albedo_afforest_100.head()

,Urban_geoid,Urban_name,Clim_zone,Coastal?,CUHI Day (25th %tile),CUHI Day (75th %tile),CUHI Day (Difference),CUHI Night (25th %tile),CUHI Night (75th %tile),CUHI Night (Difference),SUHI Day (25th %tile),SUHI Day (75th %tile),SUHI Day (Difference),SUHI Night (25th %tile),SUHI Night (75th %tile),SUHI Night (Difference)
0,280,"Abilene, TX",Temperate,False,-0.822594,-0.382081,0.440513,0.395298,0.166496,-0.228802,1.127383,0.892198,-0.235185,0.585843,0.506186,-0.079658
1,955,"Albany, OR",Temperate,False,0.119405,0.121576,0.002171,1.021048,0.415594,-0.605454,1.712134,-1.092622,-2.804756,0.916665,0.281568,-0.635098
2,1171,"Albuquerque, NM",Arid,False,0.280754,-0.032448,-0.313201,0.741243,0.588181,-0.153061,0.234785,-0.730457,-0.965242,0.906876,0.907185,0.000309
3,1279,"Alexandria, LA",Temperate,False,-0.044032,-0.094543,-0.050512,0.417584,0.151050,-0.266534,0.548167,-1.489727,-2.037894,0.586098,-0.215572,-0.801670
4,1765,"Alton, IL--MO",Temperate,False,-1.700797,-0.006134,1.694664,0.214937,0.339136,0.124199,0.997882,-2.401116,-3.398997,0.599635,0.516324,-0.083312


### Median Slopes

In [40]:
slopes = ['slope_CUHI_day', 'slope_CUHI_night', 'slope_SUHI_day', 'slope_SUHI_night']

In [41]:
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline[slopes].median(), 
    'Albedo Management': coef_result_albedo[slopes].median(), 
    '50% Afforestation':coef_result_afforest_50[slopes].median(), 
    '100% Afforestation': coef_result_afforest_100[slopes].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50[slopes].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100[slopes].median()
})
(med_results.T).to_csv('tables/median_slopes/median_coef_results.csv')
med_results.T

,slope_CUHI_day,slope_CUHI_night,slope_SUHI_day,slope_SUHI_night
Baseline,-0.009831,-0.009995,-0.076981,-0.020946
Albedo Management,-0.006066,-0.007951,-0.068228,-0.019377
50% Afforestation,-0.004639,-0.008164,-0.034255,-0.015719
100% Afforestation,-0.000766,-0.004104,-0.059709,-0.006926
Albedo Management + 50% Afforestation,-0.002586,-0.007127,-0.040305,-0.022397
Albedo Management + 100% Afforestation,0.005465,-0.007398,-0.060499,-0.009002


In [42]:
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline[slopes].median(), 
    'Albedo Management': coef_result_albedo[slopes].median(), 
    '50% Afforestation':coef_result_afforest_50[slopes].median(), 
    '100% Afforestation': coef_result_afforest_100[slopes].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50[slopes].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100[slopes].median()
})
med_results.T['slope_CUHI_night'].round(6)

Baseline                                 -0.009995
Albedo Management                        -0.007951
50% Afforestation                        -0.008164
100% Afforestation                       -0.004104
Albedo Management + 50% Afforestation    -0.007127
Albedo Management + 100% Afforestation   -0.007398
Name: slope_CUHI_night, dtype: float64

In [43]:
coef_result_baseline.groupby(['Clim_zone'])[slopes].median()

,slope_CUHI_day,slope_CUHI_night,slope_SUHI_day,slope_SUHI_night
Clim_zone,,,,
Arid,-0.012146,-0.010231,-0.031624,-0.012050
Snow,-0.013554,-0.015468,-0.115770,-0.034859
Temperate,-0.008505,-0.007500,-0.068802,-0.017332
Tropical,0.000769,0.003390,0.039779,0.012115


In [44]:
s = 'slope_CUHI_day'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
(med_results.T).to_csv('tables/median_slopes/median_coef_cuhi_day_clim_zone.csv')
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.012146,-0.013554,-0.008505,0.000769
Albedo Management,-0.006474,-0.006683,-0.005456,0.000255
50% Afforestation,-0.005721,-0.005202,-0.004135,0.000168
100% Afforestation,-0.009444,-0.004836,0.001435,0.006635
Albedo Management + 50% Afforestation,-0.007964,-0.003684,-0.001869,0.000294
Albedo Management + 100% Afforestation,-0.007344,0.009728,0.005032,0.005977


In [45]:
s = 'slope_CUHI_night'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
(med_results.T).to_csv('tables/median_slopes/median_coef_cuhi_night_clim_zone.csv')
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.010231,-0.015468,-0.007500,0.003390
Albedo Management,-0.007696,-0.012054,-0.006514,0.003247
50% Afforestation,-0.006109,-0.012902,-0.006564,-0.003885
100% Afforestation,-0.006056,-0.005813,-0.003132,0.005367
Albedo Management + 50% Afforestation,-0.007104,-0.007917,-0.006431,-0.001045
Albedo Management + 100% Afforestation,-0.004652,-0.011549,-0.006387,0.003743


In [46]:
s = 'slope_SUHI_day'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
(med_results.T).to_csv('tables/median_slopes/median_coef_suhi_day_clim_zone.csv')
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.031624,-0.115770,-0.068802,0.039779
Albedo Management,-0.032312,-0.094246,-0.063815,0.017054
50% Afforestation,-0.023807,-0.048721,-0.031733,0.026043
100% Afforestation,-0.023722,-0.089975,-0.054574,0.052872
Albedo Management + 50% Afforestation,-0.028733,-0.046659,-0.038456,0.034570
Albedo Management + 100% Afforestation,-0.030941,-0.081753,-0.058501,0.063458


In [47]:
s = 'slope_SUHI_night'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
(med_results.T).to_csv('tables/median_slopes/median_coef_suhi_night_clim_zone.csv')
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.012050,-0.034859,-0.017332,0.012115
Albedo Management,-0.010021,-0.027458,-0.017182,0.001270
50% Afforestation,-0.008678,-0.022645,-0.013815,0.000613
100% Afforestation,-0.006198,-0.011633,-0.005897,0.010406
Albedo Management + 50% Afforestation,-0.008249,-0.031043,-0.020650,-0.004015
Albedo Management + 100% Afforestation,-0.005749,-0.008193,-0.009527,0.005979


### Descriptive Table Helper Function

In [48]:
def set_des_results(slope):
    des_results = pd.DataFrame({
        'Baseline': coef_result_baseline[slope].describe(), 
        'Albedo Management': coef_result_albedo[slope].describe(), 
        '50% Afforestation':coef_result_afforest_50[slope].describe(), 
        '100% Afforestation': coef_result_afforest_100[slope].describe(), 
        'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50[slope].describe(), 
        'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100[slope].describe()
    })
    return des_results.T

### CUHI Day

In [49]:
set_des_results('slope_CUHI_day')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.013725,0.017729,-0.131985,-0.017676,-0.009831,-0.004397,0.072990
Albedo Management,493.0,-0.008880,0.013967,-0.110230,-0.010920,-0.006066,-0.002683,0.062278
50% Afforestation,493.0,-0.007778,0.021342,-0.116472,-0.015018,-0.004639,0.003381,0.057246
100% Afforestation,493.0,-0.001711,0.029565,-0.124031,-0.015292,-0.000766,0.012888,0.192382
Albedo Management + 50% Afforestation,493.0,-0.006604,0.019228,-0.150876,-0.009052,-0.002586,0.001660,0.043726
Albedo Management + 100% Afforestation,493.0,0.005103,0.029101,-0.125587,-0.007697,0.005465,0.020681,0.151003


### CUHI Night

In [50]:
set_des_results('slope_CUHI_night')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.011489,0.015400,-0.087587,-0.018422,-0.009995,-0.003823,0.114680
Albedo Management,493.0,-0.009798,0.012397,-0.060760,-0.014538,-0.007951,-0.003361,0.065065
50% Afforestation,493.0,-0.011998,0.018453,-0.086263,-0.016890,-0.008164,-0.002192,0.122584
100% Afforestation,493.0,-0.005929,0.013662,-0.077012,-0.009044,-0.004104,-0.000131,0.130552
Albedo Management + 50% Afforestation,493.0,-0.011712,0.019020,-0.134665,-0.014130,-0.007127,-0.002682,0.072109
Albedo Management + 100% Afforestation,493.0,-0.009793,0.014897,-0.108190,-0.014896,-0.007398,-0.002921,0.057965


### SUHI Night

In [51]:
set_des_results('slope_SUHI_day')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.086144,0.076465,-0.363049,-0.129956,-0.076981,-0.038142,0.214776
Albedo Management,493.0,-0.075394,0.061064,-0.281453,-0.108732,-0.068228,-0.037880,0.202342
50% Afforestation,493.0,-0.041730,0.053165,-0.260488,-0.065263,-0.034255,-0.016296,0.335352
100% Afforestation,493.0,-0.068073,0.067911,-0.374760,-0.101524,-0.059709,-0.028089,0.279096
Albedo Management + 50% Afforestation,493.0,-0.045042,0.051894,-0.247448,-0.069647,-0.040305,-0.019416,0.321343
Albedo Management + 100% Afforestation,493.0,-0.066763,0.066630,-0.385497,-0.098775,-0.060499,-0.026683,0.268553


### SUHI Night

In [52]:
set_des_results('slope_SUHI_night')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.023901,0.027963,-0.158291,-0.038910,-0.020946,-0.007775,0.155813
Albedo Management,493.0,-0.021154,0.022336,-0.109194,-0.032035,-0.019377,-0.009106,0.101569
50% Afforestation,493.0,-0.018944,0.025643,-0.161125,-0.030847,-0.015719,-0.005362,0.159835
100% Afforestation,493.0,-0.008269,0.015522,-0.069036,-0.015681,-0.006926,-0.000150,0.090718
Albedo Management + 50% Afforestation,493.0,-0.025825,0.026455,-0.168254,-0.038831,-0.022397,-0.010449,0.073888
Albedo Management + 100% Afforestation,493.0,-0.011379,0.016195,-0.078029,-0.019775,-0.009002,-0.002482,0.089680


In [53]:
coef_result_baseline.head()

,Urban_geoid,slope_CUHI_day,slope_CUHI_night,slope_SUHI_day,slope_SUHI_night,Urban_name,Clim_zone,Coastal?
0,199,-0.014211,-0.021060,-0.030373,-0.027808,"Aberdeen--Bel Air South--Bel Air North, MD",Temperate,True
1,280,-0.008774,-0.009990,-0.001643,-0.007958,"Abilene, TX",Temperate,False
2,631,0.000769,0.001019,0.133467,-0.021660,"Aguadilla--Isabela--San Sebastián, PR",Tropical,True
3,766,-0.011416,-0.010439,-0.106883,-0.031509,"Akron, OH",Snow,False
4,901,-0.013309,-0.009631,-0.145180,-0.032123,"Albany, GA",Temperate,False


In [54]:
coef_result_baseline.to_csv('tables/coef/baseline.csv', index=False)

In [55]:
coef_result_albedo.to_csv('tables/coef/albedo.csv', index=False)

In [56]:
coef_result_afforest_50.to_csv('tables/coef/afforest_50.csv', index=False)

In [57]:
coef_result_afforest_100.to_csv('tables/coef/afforest_100.csv', index=False)

In [58]:
coef_result_albedo_afforest_50.to_csv('tables/coef/albedo_afforest_50.csv', index=False)

In [59]:
coef_result_albedo_afforest_100.to_csv('tables/coef/albedo_afforest_100.csv', index=False)